# Player Facts Preparation
## Collecting NHL Players' Facts from EliteProspects
This step gathers player facts data for downstream NHL prediction.

- Collect league-wide player stats for a given season from
https://eliteprospects.com/league/{league}/stats/{season}?page={n}

- Extract metadata, including individual player profile links.

- Build a unique player list from seasons 2000–2025 by storing profile links in a set.

- Scrape each player’s page to retrieve their first 5 years of NHL stats, filtered by seasons where they played more than 30 games.

- Gather pre-NHL stats for each player (junior, college, international, etc.).

## Import Libraries

In [1]:
import eliteprospects_scraper_api as ep
import pandas as pd
import os
import time
import random

### Collecting NHL Players' Metadata from EliteProspects

In [2]:
if not os.path.exists('./data/nhl/nhl_players_metadata.csv'):
    # Collect players from 2000-2025 season
    for i in range(0, 25):
        season = f'20{str(i).zfill(2)}-20{str(i + 1).zfill(2)}'
        print(f'Scraping {season}')
        nhl_players = ep.get_season_roster("nhl", season)
        nhl_players.to_csv(f'./data/nhl/players/nhl_players_{season}.csv', index=False, encoding='utf-8-sig')
        print(f'Finished scraping {season}')
else:
    print('NHL players metadata already exists. Skipping scraping.')

NHL players metadata already exists. Skipping scraping.


In [3]:
if not os.path.exists('./data/nhl/nhl_players_metadata.csv'):
    # Collect players' metadata from 2000-2025 season
    nhl_players_metadata = pd.DataFrame()
    for i in range(0, 25):
        season = f'20{str(i).zfill(2)}-20{str(i + 1).zfill(2)}'
        nhl_players = pd.read_csv(f'./data/nhl/players/nhl_players_{season}.csv')
        nhl_players_metadata = pd.concat([nhl_players_metadata, ep.get_players_metadata(nhl_players)]).reset_index(drop=True)
        # Remove duplicates
        nhl_players_metadata = nhl_players_metadata.drop_duplicates(subset=['player_name']).reset_index(drop=True)
        print(f'Finished Merging Metadata for {season}')
else:
    print('NHL players metadata already exists. Skipping merging.')
    nhl_players_metadata = pd.read_csv('./data/nhl/nhl_players_metadata.csv')


NHL players metadata already exists. Skipping merging.


### Testing API

In [ ]:
cale_makar_metadata = nhl_players_metadata.iloc[3188]
cale_makar_metadata

In [ ]:
cale_makar_facts = ep.get_player_facts(cale_makar_metadata)

In [ ]:
cale_makar_facts

### Collecting NHL Players' Facts from EliteProspects

In [4]:
# Define output file
output_path = './data/nhl/facts/nhl_players_facts.csv'

In [5]:
def get_players_stats_by_batch(players_to_scrape):
    curr_len = len(players_to_scrape)
    fail_count = 0

    for i in range(curr_len):
        player_metadata = players_to_scrape.iloc[i]
        player_name = player_metadata['player_name']
        player_url = player_metadata['link']
        print(f"\n [{i + 1}] Collecting facts for {player_name} at {player_url}")

        try:
            player_stats = ep.get_player_facts(player_metadata)

            # Write to CSV file
            if os.path.exists(output_path):
                player_stats.to_csv(output_path, mode='a', header=False, index=False, encoding='utf-8-sig')
            else:
                player_stats.to_csv(output_path, index=False, encoding='utf-8-sig')
            print(f'Successfully scraped facts for {player_name}')

            # Print Fail Rate
            print(f'Failed rate: {fail_count / (i + 1):.2f}')

            # Add random sleep to prevent getting blocked
            if i < curr_len - 1:
                sleep_time = random.uniform(10, 120)
                print(f"Sleep for {sleep_time / 60:.2f} minutes to prevent getting blocked")
                time.sleep(sleep_time)
        except Exception as e:
            print(f"Failed to get facts for {player_name}: {e}")

            fail_count += 1

            if i < curr_len - 1:
                # Sleep for 15-60 seconds before trying the next player
                sleep_time = random.uniform(15, 60)
                print(f"Sleeping for {sleep_time / 60:.2f} seconds before trying the next player")
                time.sleep(sleep_time)

### Fetch Player Facts in Batches

In [ ]:
# Fetch 0 to 10 players' facts
curr_players_metadata = nhl_players_metadata[0:10]
get_players_stats_by_batch(curr_players_metadata)

In [ ]:
# Fetch 10 to 50 players' facts
curr_players_metadata = nhl_players_metadata[10:50]
get_players_stats_by_batch(curr_players_metadata)

In [ ]:
# Fetch 50 to 100 players' facts
curr_players_metadata = nhl_players_metadata[50:100]
get_players_stats_by_batch(curr_players_metadata)

In [ ]:
# Fetch 100 to 150 players' facts
curr_players_metadata = nhl_players_metadata[100:150]
get_players_stats_by_batch(curr_players_metadata)

In [ ]:
# Fetch 150 to 200 players' facts
curr_players_metadata = nhl_players_metadata[150:200]
get_players_stats_by_batch(curr_players_metadata)

In [ ]:
# Fetch 200 to 250 players' facts
curr_players_metadata = nhl_players_metadata[200:250]
get_players_stats_by_batch(curr_players_metadata)

#### 07/26/2025 -- Midnight

In [11]:
# Fetch 250 to 300 players' facts
curr_players_metadata = nhl_players_metadata[250:300]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Dean McAmmond at https://www.eliteprospects.com/player/8972/dean-mcammond
Failed to extract player types - No player types found.
Successfully scraped facts for Dean McAmmond
Failed rate: 0.00
Sleep for 1.89 minutes to prevent getting blocked

 [2] Collecting facts for Donald Brashear at https://www.eliteprospects.com/player/9094/donald-brashear
Successfully scraped facts for Donald Brashear
Failed rate: 0.00
Sleep for 1.64 minutes to prevent getting blocked

 [3] Collecting facts for Mattias Öhlund at https://www.eliteprospects.com/player/749/mattias-ohlund
Failed to extract player types - No player types found.
Successfully scraped facts for Mattias Öhlund
Failed rate: 0.00
Sleep for 1.65 minutes to prevent getting blocked

 [4] Collecting facts for Robert Svehla at https://www.eliteprospects.com/player/1909/robert-svehla
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Robert Svehla
Failed rate: 

In [12]:
# Fetch 300 to 350 players' facts
curr_players_metadata = nhl_players_metadata[300:350]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Andrei Markov at https://www.eliteprospects.com/player/8579/andrei-markov
Successfully scraped facts for Andrei Markov
Failed rate: 0.00
Sleep for 0.26 minutes to prevent getting blocked

 [2] Collecting facts for Mike Eastwood at https://www.eliteprospects.com/player/69415/mike-eastwood
Failed to extract player types - No player types found.
Successfully scraped facts for Mike Eastwood
Failed rate: 0.00
Sleep for 1.76 minutes to prevent getting blocked

 [3] Collecting facts for Jan Bulis at https://www.eliteprospects.com/player/8368/jan-bulis
Failed to extract player types - No player types found.
Successfully scraped facts for Jan Bulis
Failed rate: 0.00
Sleep for 0.66 minutes to prevent getting blocked

 [4] Collecting facts for Len Barrie at https://www.eliteprospects.com/player/31290/len-barrie
Failed to extract player types - No player types found.
Successfully scraped facts for Len Barrie
Failed rate: 0.00
Sleep for 1.69 minutes to prevent getting blo

In [13]:
# Fetch 350 to 400 players' facts
curr_players_metadata = nhl_players_metadata[350:400]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Jeff Shantz at https://www.eliteprospects.com/player/12798/jeff-shantz
Failed to extract player types - No player types found.
Successfully scraped facts for Jeff Shantz
Failed rate: 0.00
Sleep for 0.62 minutes to prevent getting blocked

 [2] Collecting facts for Jason Smith at https://www.eliteprospects.com/player/8883/jason-smith
Failed to extract player types - No player types found.
Successfully scraped facts for Jason Smith
Failed rate: 0.00
Sleep for 1.62 minutes to prevent getting blocked

 [3] Collecting facts for Andrei Zyuzin at https://www.eliteprospects.com/player/8608/andrei-zyuzin
Failed to extract player types - No player types found.
Successfully scraped facts for Andrei Zyuzin
Failed rate: 0.00
Sleep for 0.19 minutes to prevent getting blocked

 [4] Collecting facts for Richard Matvichuk at https://www.eliteprospects.com/player/8970/richard-matvichuk
Failed to extract player types - No player types found.
Successfully scraped facts for Richa

In [14]:
# Fetch 400 to 450 players' facts
curr_players_metadata = nhl_players_metadata[400:450]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Bill Houlder at https://www.eliteprospects.com/player/67298/bill-houlder
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Bill Houlder
Failed rate: 0.00
Sleep for 1.50 minutes to prevent getting blocked

 [2] Collecting facts for Ossi Väänänen at https://www.eliteprospects.com/player/2685/ossi-vaananen
Successfully scraped facts for Ossi Väänänen
Failed rate: 0.00
Sleep for 0.39 minutes to prevent getting blocked

 [3] Collecting facts for Jonathan Girard at https://www.eliteprospects.com/player/67360/jonathan-girard
Failed to extract player types - No player types found.
Successfully scraped facts for Jonathan Girard
Failed rate: 0.00
Sleep for 0.46 minutes to prevent getting blocked

 [4] Collecting facts for Daniil Markov at https://www.eliteprospects.com/player/8613/daniil-markov
Description not found.
Successfully scraped facts for Daniil Markov
Failed rate: 0.00
Sleep for 0.40 minutes to preve

In [15]:
# Fetch 450 to 500 players' facts
curr_players_metadata = nhl_players_metadata[450:500]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Matthew Barnaby at https://www.eliteprospects.com/player/9073/matthew-barnaby
Successfully scraped facts for Matthew Barnaby
Failed rate: 0.00
Sleep for 1.13 minutes to prevent getting blocked

 [2] Collecting facts for Marty Reasoner at https://www.eliteprospects.com/player/8708/marty-reasoner
Failed to extract player types - No player types found.
Successfully scraped facts for Marty Reasoner
Failed rate: 0.00
Sleep for 0.92 minutes to prevent getting blocked

 [3] Collecting facts for James Patrick at https://www.eliteprospects.com/player/32586/james-patrick
Failed to extract player types - No player types found.
Successfully scraped facts for James Patrick
Failed rate: 0.00
Sleep for 0.97 minutes to prevent getting blocked

 [4] Collecting facts for Cam Stewart at https://www.eliteprospects.com/player/25626/cam-stewart
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Cam Stewart
Failed rate: 0.0

In [16]:
# Fetch 500 to 550 players' facts
curr_players_metadata = nhl_players_metadata[500:550]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Nathan Dempsey at https://www.eliteprospects.com/player/8741/nathan-dempsey
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Nathan Dempsey
Failed rate: 0.00
Sleep for 1.09 minutes to prevent getting blocked

 [2] Collecting facts for Willie Mitchell at https://www.eliteprospects.com/player/8548/willie-mitchell
Description not found.
Successfully scraped facts for Willie Mitchell
Failed rate: 0.00
Sleep for 0.27 minutes to prevent getting blocked

 [3] Collecting facts for Kirk Muller at https://www.eliteprospects.com/player/22413/kirk-muller
[ERROR] Failed to get facts for Kirk Muller: HTTPConnectionPool(host='localhost', port=62517): Read timed out. (read timeout=120)
Successfully scraped facts for Kirk Muller
Failed rate: 0.00
Sleep for 1.35 minutes to prevent getting blocked

 [4] Collecting facts for Dave Reid at https://www.eliteprospects.com/player/67436/dave-reid
Failed to extract player typ

In [17]:
# Fetch 550 to 600 players' facts
curr_players_metadata = nhl_players_metadata[550:600]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Ian Moran at https://www.eliteprospects.com/player/5085/ian-moran
Failed to extract player types - No player types found.
Successfully scraped facts for Ian Moran
Failed rate: 0.00
Sleep for 1.26 minutes to prevent getting blocked

 [2] Collecting facts for Marc Chouinard at https://www.eliteprospects.com/player/8547/marc-chouinard
Failed to extract player types - No player types found.
Successfully scraped facts for Marc Chouinard
Failed rate: 0.00
Sleep for 1.37 minutes to prevent getting blocked

 [3] Collecting facts for Juha Lind at https://www.eliteprospects.com/player/259/juha-lind
Successfully scraped facts for Juha Lind
Failed rate: 0.00
Sleep for 1.18 minutes to prevent getting blocked

 [4] Collecting facts for Jason Marshall at https://www.eliteprospects.com/player/8533/jason-marshall
Failed to extract player types - No player types found.
Successfully scraped facts for Jason Marshall
Failed rate: 0.00
Sleep for 0.92 minutes to prevent getting blo

In [18]:
# Fetch 600 to 650 players' facts
curr_players_metadata = nhl_players_metadata[600:650]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Sylvain Blouin at https://www.eliteprospects.com/player/68876/sylvain-blouin
Description not found.
Successfully scraped facts for Sylvain Blouin
Failed rate: 0.00
Sleep for 0.79 minutes to prevent getting blocked

 [2] Collecting facts for Blake Sloan at https://www.eliteprospects.com/player/6811/blake-sloan
Failed to extract player types - No player types found.
Successfully scraped facts for Blake Sloan
Failed rate: 0.00
Sleep for 1.86 minutes to prevent getting blocked

 [3] Collecting facts for Shane Hnidy at https://www.eliteprospects.com/player/8560/shane-hnidy
Failed to extract player types - No player types found.
Successfully scraped facts for Shane Hnidy
Failed rate: 0.00
Sleep for 0.70 minutes to prevent getting blocked

 [4] Collecting facts for Stu Grimson at https://www.eliteprospects.com/player/43441/stu-grimson
Description not found.
Successfully scraped facts for Stu Grimson
Failed rate: 0.00
Sleep for 1.10 minutes to prevent getting blocked

In [19]:
# Fetch 650 to 700 players' facts
curr_players_metadata = nhl_players_metadata[650:700]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Ron Sutter at https://www.eliteprospects.com/player/29015/ron-sutter
Failed to extract player types - No player types found.
Successfully scraped facts for Ron Sutter
Failed rate: 0.00
Sleep for 0.97 minutes to prevent getting blocked

 [2] Collecting facts for Alexei Ponikarovsky at https://www.eliteprospects.com/player/8658/alexei-ponikarovsky
Failed to extract player types - No player types found.
Successfully scraped facts for Alexei Ponikarovsky
Failed rate: 0.00
Sleep for 1.06 minutes to prevent getting blocked

 [3] Collecting facts for Jamie Allison at https://www.eliteprospects.com/player/9063/jamie-allison
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Jamie Allison
Failed rate: 0.00
Sleep for 0.30 minutes to prevent getting blocked

 [4] Collecting facts for Brad Brown at https://www.eliteprospects.com/player/11823/brad-brown
Failed to extract player types - No player types found.
Succe

In [26]:
# Fetch 700 to 750 players' facts
curr_players_metadata = nhl_players_metadata[700:750]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Mark Greig at https://www.eliteprospects.com/player/54582/mark-greig
[ERROR] Failed to get facts for Mark Greig: HTTPConnectionPool(host='localhost', port=54411): Read timed out. (read timeout=120)
Successfully scraped facts for Mark Greig
Failed rate: 0.00
Sleep for 1.21 minutes to prevent getting blocked

 [2] Collecting facts for Dmitry Afanasenkov at https://www.eliteprospects.com/player/8634/dmitry-afanasenkov
Failed to extract player types - No player types found.
Successfully scraped facts for Dmitry Afanasenkov
Failed rate: 0.00
Sleep for 1.46 minutes to prevent getting blocked

 [3] Collecting facts for Antti Aalto at https://www.eliteprospects.com/player/2703/antti-aalto
Successfully scraped facts for Antti Aalto
Failed rate: 0.00
Sleep for 0.87 minutes to prevent getting blocked

 [4] Collecting facts for Yuri Butsayev at https://www.eliteprospects.com/player/18507/yuri-butsayev
Failed to extract player types - No player types found.
Successfully s

In [21]:
# Fetch 750 to 800 players' facts
curr_players_metadata = nhl_players_metadata[750:800]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Sean Haggerty at https://www.eliteprospects.com/player/31356/sean-haggerty
Failed to extract player types - No player types found.
Successfully scraped facts for Sean Haggerty
Failed rate: 0.00
Sleep for 0.46 minutes to prevent getting blocked

 [2] Collecting facts for Don MacLean at https://www.eliteprospects.com/player/8953/don-maclean
Successfully scraped facts for Don MacLean
Failed rate: 0.00
Sleep for 0.59 minutes to prevent getting blocked

 [3] Collecting facts for Ed Ward at https://www.eliteprospects.com/player/568/ed-ward
Failed to extract player types - No player types found.
Successfully scraped facts for Ed Ward
Failed rate: 0.00
Sleep for 0.66 minutes to prevent getting blocked

 [4] Collecting facts for Darryl Shannon at https://www.eliteprospects.com/player/67424/darryl-shannon
Failed to extract player types - No player types found.
Successfully scraped facts for Darryl Shannon
Failed rate: 0.00
Sleep for 1.48 minutes to prevent getting bloc

In [22]:
# Fetch 800 to 850 players' facts
curr_players_metadata = nhl_players_metadata[800:850]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Paul Kruse at https://www.eliteprospects.com/player/27139/paul-kruse
Description not found.
Successfully scraped facts for Paul Kruse
Failed rate: 0.00
Sleep for 0.19 minutes to prevent getting blocked

 [2] Collecting facts for Sean Selmser at https://www.eliteprospects.com/player/15579/sean-selmser
Failed to extract player types - No player types found.
Successfully scraped facts for Sean Selmser
Failed rate: 0.00
Sleep for 1.78 minutes to prevent getting blocked

 [3] Collecting facts for Jody Shelley at https://www.eliteprospects.com/player/8722/jody-shelley
Successfully scraped facts for Jody Shelley
Failed rate: 0.00
Sleep for 1.83 minutes to prevent getting blocked

 [4] Collecting facts for Jean-Pierre Vigier at https://www.eliteprospects.com/player/8841/jean-pierre-vigier
Failed to extract player types - No player types found.
Successfully scraped facts for Jean-Pierre Vigier
Failed rate: 0.00
Sleep for 1.60 minutes to prevent getting blocked

 [5] C

In [23]:
# Fetch 850 to 900 players' facts
curr_players_metadata = nhl_players_metadata[850:900]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for David Emma at https://www.eliteprospects.com/player/24188/david-emma
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for David Emma
Failed rate: 0.00
Sleep for 0.65 minutes to prevent getting blocked

 [2] Collecting facts for Bryan Allen at https://www.eliteprospects.com/player/8767/bryan-allen
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Bryan Allen
Failed rate: 0.00
Sleep for 1.93 minutes to prevent getting blocked

 [3] Collecting facts for Matt Higgins at https://www.eliteprospects.com/player/13398/matt-higgins
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Matt Higgins
Failed rate: 0.00
Sleep for 1.38 minutes to prevent getting blocked

 [4] Collecting facts for Tony Tuzzolino at https://www.eliteprospects.com/player/7283/tony-tuzzolino
Failed to extract player types - N

In [24]:
# Fetch 900 to 950 players' facts
curr_players_metadata = nhl_players_metadata[900:950]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Jiri Dopita at https://www.eliteprospects.com/player/10873/jiri-dopita
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Jiri Dopita
Failed rate: 0.00
Sleep for 0.98 minutes to prevent getting blocked

 [2] Collecting facts for Zdeno Cíger at https://www.eliteprospects.com/player/69982/zdeno-ciger
Failed to extract player types - No player types found.
Successfully scraped facts for Zdeno Cíger
Failed rate: 0.00
Sleep for 0.56 minutes to prevent getting blocked

 [3] Collecting facts for Richard Park at https://www.eliteprospects.com/player/5214/richard-park
Failed to extract player types - No player types found.
Successfully scraped facts for Richard Park
Failed rate: 0.00
Sleep for 0.39 minutes to prevent getting blocked

 [4] Collecting facts for Andreas Johansson at https://www.eliteprospects.com/player/696/andreas-johansson
Failed to extract player types - No player types found.
Successfully scr

In [25]:
# Fetch 950 to 1000 players' facts
curr_players_metadata = nhl_players_metadata[950:1000]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Kelly Fairchild at https://www.eliteprospects.com/player/15424/kelly-fairchild
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Kelly Fairchild
Failed rate: 0.00
Sleep for 0.25 minutes to prevent getting blocked

 [2] Collecting facts for Stephen Weiss at https://www.eliteprospects.com/player/8817/stephen-weiss
Failed to extract player types - No player types found.
Successfully scraped facts for Stephen Weiss
Failed rate: 0.00
Sleep for 0.66 minutes to prevent getting blocked

 [3] Collecting facts for Craig MacDonald at https://www.eliteprospects.com/player/8952/craig-macdonald
Failed to extract player types - No player types found.
Successfully scraped facts for Craig MacDonald
Failed rate: 0.00
Sleep for 1.03 minutes to prevent getting blocked

 [4] Collecting facts for Andrej Nedorost at https://www.eliteprospects.com/player/9672/andrej-nedorost
Failed to extract player types - No player types 

In [27]:
# Fetch 1000 to 1050 players' facts
curr_players_metadata = nhl_players_metadata[1000:1050]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Henrik Tallinder at https://www.eliteprospects.com/player/1955/henrik-tallinder
Successfully scraped facts for Henrik Tallinder
Failed rate: 0.00
Sleep for 1.16 minutes to prevent getting blocked

 [2] Collecting facts for Brad Leeb at https://www.eliteprospects.com/player/9529/brad-leeb
Failed to extract player types - No player types found.
Successfully scraped facts for Brad Leeb
Failed rate: 0.00
Sleep for 1.08 minutes to prevent getting blocked

 [3] Collecting facts for Alexei Tezikov at https://www.eliteprospects.com/player/13529/alexei-tezikov
Failed to extract player types - No player types found.
Successfully scraped facts for Alexei Tezikov
Failed rate: 0.00
Sleep for 0.30 minutes to prevent getting blocked

 [4] Collecting facts for Alan Letang at https://www.eliteprospects.com/player/19450/alan-letang
Failed to extract player types - No player types found.
Successfully scraped facts for Alan Letang
Failed rate: 0.00
Sleep for 1.16 minutes to prev

In [28]:
# Fetch 1050 to 1100 players' facts
curr_players_metadata = nhl_players_metadata[1050:1100]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Lasse Pirjetä at https://www.eliteprospects.com/player/1798/lasse-pirjeta
Successfully scraped facts for Lasse Pirjetä
Failed rate: 0.00
Sleep for 1.30 minutes to prevent getting blocked

 [2] Collecting facts for Jason Spezza at https://www.eliteprospects.com/player/3661/jason-spezza
Successfully scraped facts for Jason Spezza
Failed rate: 0.00
Sleep for 1.69 minutes to prevent getting blocked

 [3] Collecting facts for Pierre-Marc Bouchard at https://www.eliteprospects.com/player/9086/pierre-marc-bouchard
Failed to extract player types - No player types found.
Successfully scraped facts for Pierre-Marc Bouchard
Failed rate: 0.00
Sleep for 1.85 minutes to prevent getting blocked

 [4] Collecting facts for Jim Fahey at https://www.eliteprospects.com/player/9150/jim-fahey
Failed to extract player types - No player types found.
Successfully scraped facts for Jim Fahey
Failed rate: 0.00
Sleep for 0.22 minutes to prevent getting blocked

 [5] Collecting facts for

In [29]:
# Fetch 1100 to 1150 players' facts
curr_players_metadata = nhl_players_metadata[1100:1150]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Shawn Thornton at https://www.eliteprospects.com/player/11339/shawn-thornton
Description not found.
Successfully scraped facts for Shawn Thornton
Failed rate: 0.00
Sleep for 0.60 minutes to prevent getting blocked

 [2] Collecting facts for Jason King at https://www.eliteprospects.com/player/9920/jason-king
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Jason King
Failed rate: 0.00
Sleep for 0.20 minutes to prevent getting blocked

 [3] Collecting facts for Chris Schmidt at https://www.eliteprospects.com/player/13963/chris-schmidt
Failed to extract player types - No player types found.
Successfully scraped facts for Chris Schmidt
Failed rate: 0.00
Sleep for 1.38 minutes to prevent getting blocked

 [4] Collecting facts for Garnet Exelby at https://www.eliteprospects.com/player/8558/garnet-exelby
Successfully scraped facts for Garnet Exelby
Failed rate: 0.00
Sleep for 0.60 minutes to prevent gettin

In [30]:
# Fetch 1150 to 1200 players' facts
curr_players_metadata = nhl_players_metadata[1150:1200]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Kent McDonell at https://www.eliteprospects.com/player/8745/kent-mcdonell
[ERROR] Failed to get facts for Kent McDonell: HTTPConnectionPool(host='localhost', port=51592): Read timed out. (read timeout=120)
Successfully scraped facts for Kent McDonell
Failed rate: 0.00
Sleep for 1.03 minutes to prevent getting blocked

 [2] Collecting facts for Patrick Sharp at https://www.eliteprospects.com/player/8880/patrick-sharp
Failed to extract player types - No player types found.
Successfully scraped facts for Patrick Sharp
Failed rate: 0.00
Sleep for 1.94 minutes to prevent getting blocked

 [3] Collecting facts for Alain Nasreddine at https://www.eliteprospects.com/player/11042/alain-nasreddine
Failed to extract player types - No player types found.
Successfully scraped facts for Alain Nasreddine
Failed rate: 0.00
Sleep for 1.18 minutes to prevent getting blocked

 [4] Collecting facts for Ross Lupaschuk at https://www.eliteprospects.com/player/5979/ross-lupaschuk
F

In [31]:
# Fetch 1200 to 1250 players' facts
curr_players_metadata = nhl_players_metadata[1200:1250]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Fyodor Tyutin at https://www.eliteprospects.com/player/8691/fyodor-tyutin
Failed to extract player types - No player types found.
Successfully scraped facts for Fyodor Tyutin
Failed rate: 0.00
Sleep for 0.54 minutes to prevent getting blocked

 [2] Collecting facts for Tim Gleason at https://www.eliteprospects.com/player/9162/tim-gleason
Failed to extract player types - No player types found.
Successfully scraped facts for Tim Gleason
Failed rate: 0.00
Sleep for 1.41 minutes to prevent getting blocked

 [3] Collecting facts for Travis Moen at https://www.eliteprospects.com/player/8535/travis-moen
Failed to extract player types - No player types found.
Successfully scraped facts for Travis Moen
Failed rate: 0.00
Sleep for 1.70 minutes to prevent getting blocked

 [4] Collecting facts for Jed Ortmeyer at https://www.eliteprospects.com/player/8942/jed-ortmeyer
Failed to extract player types - No player types found.
Successfully scraped facts for Jed Ortmeyer
Fai

In [32]:
# Fetch 1250 to 1300 players' facts
curr_players_metadata = nhl_players_metadata[1250:1300]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Dan Fritsche at https://www.eliteprospects.com/player/9434/dan-fritsche
Successfully scraped facts for Dan Fritsche
Failed rate: 0.00
Sleep for 1.70 minutes to prevent getting blocked

 [2] Collecting facts for Garth Murray at https://www.eliteprospects.com/player/8956/garth-murray
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Garth Murray
Failed rate: 0.00
Sleep for 0.35 minutes to prevent getting blocked

 [3] Collecting facts for Noah Clarke at https://www.eliteprospects.com/player/10321/noah-clarke
Failed to extract player types - No player types found.
Successfully scraped facts for Noah Clarke
Failed rate: 0.00
Sleep for 0.34 minutes to prevent getting blocked

 [4] Collecting facts for Pierre Hedin at https://www.eliteprospects.com/player/218/pierre-hedin
Failed to extract player types - No player types found.
Successfully scraped facts for Pierre Hedin
Failed rate: 0.00
Sleep for 0.91 min

In [33]:
# Fetch 1300 to 1350 players' facts
curr_players_metadata = nhl_players_metadata[1300:1350]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Mike Pandolfo at https://www.eliteprospects.com/player/14421/mike-pandolfo
Failed to extract player types - No player types found.
Successfully scraped facts for Mike Pandolfo
Failed rate: 0.00
Sleep for 0.96 minutes to prevent getting blocked

 [2] Collecting facts for Mikhail Kuleshov at https://www.eliteprospects.com/player/39446/mikhail-kuleshov
Failed to extract player types - No player types found.
Successfully scraped facts for Mikhail Kuleshov
Failed rate: 0.00
Sleep for 1.57 minutes to prevent getting blocked

 [3] Collecting facts for Benoit Dusablon at https://www.eliteprospects.com/player/60357/benoit-dusablon
Failed to extract player types - No player types found.
Successfully scraped facts for Benoit Dusablon
Failed rate: 0.00
Sleep for 1.92 minutes to prevent getting blocked

 [4] Collecting facts for Goran Bezina at https://www.eliteprospects.com/player/12299/goran-bezina
Failed to extract player types - No player types found.
Successfully scr

In [34]:
# Fetch 1350 to 1400 players' facts
curr_players_metadata = nhl_players_metadata[1350:1400]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Jay McClement at https://www.eliteprospects.com/player/9517/jay-mcclement
Failed to extract player types - No player types found.
Successfully scraped facts for Jay McClement
Failed rate: 0.00
Sleep for 0.48 minutes to prevent getting blocked

 [2] Collecting facts for Corey Perry at https://www.eliteprospects.com/player/8541/corey-perry
Successfully scraped facts for Corey Perry
Failed rate: 0.00
Sleep for 0.99 minutes to prevent getting blocked

 [3] Collecting facts for Dennis Wideman at https://www.eliteprospects.com/player/8826/dennis-wideman
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Dennis Wideman
Failed rate: 0.00
Sleep for 1.69 minutes to prevent getting blocked

 [4] Collecting facts for Jaroslav Balastik at https://www.eliteprospects.com/player/8618/jaroslav-balastik
Failed to extract player types - No player types found.
Successfully scraped facts for Jaroslav Balastik
Failed rate:

In [35]:
# Fetch 1400 to 1450 players' facts
curr_players_metadata = nhl_players_metadata[1400:1450]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Geoff Platt at https://www.eliteprospects.com/player/9469/geoff-platt
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Geoff Platt
Failed rate: 0.00
Sleep for 1.78 minutes to prevent getting blocked

 [2] Collecting facts for Gilbert Brulé at https://www.eliteprospects.com/player/10235/gilbert-brule
Failed to extract player types - No player types found.
Successfully scraped facts for Gilbert Brulé
Failed rate: 0.00
Sleep for 1.59 minutes to prevent getting blocked

 [3] Collecting facts for Noah Welch at https://www.eliteprospects.com/player/10323/noah-welch
Failed to extract player types - No player types found.
Successfully scraped facts for Noah Welch
Failed rate: 0.00
Sleep for 1.65 minutes to prevent getting blocked

 [4] Collecting facts for Yan Stastny at https://www.eliteprospects.com/player/8799/yan-stastny
Failed to extract player types - No player types found.
Successfully scraped facts 

In [36]:
# Fetch 1450 to 1500 players' facts
curr_players_metadata = nhl_players_metadata[1450:1500]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Greg Jacina at https://www.eliteprospects.com/player/10248/greg-jacina
Failed to extract player types - No player types found.
Successfully scraped facts for Greg Jacina
Failed rate: 0.00
Sleep for 0.69 minutes to prevent getting blocked

 [2] Collecting facts for Matt Ryan at https://www.eliteprospects.com/player/14334/matt-ryan
Failed to extract player types - No player types found.
Successfully scraped facts for Matt Ryan
Failed rate: 0.00
Sleep for 1.36 minutes to prevent getting blocked

 [3] Collecting facts for Nick Tarnasky at https://www.eliteprospects.com/player/8846/nick-tarnasky
Failed to extract player types - No player types found.
Successfully scraped facts for Nick Tarnasky
Failed rate: 0.00
Sleep for 1.12 minutes to prevent getting blocked

 [4] Collecting facts for Andy Wozniewski at https://www.eliteprospects.com/player/8543/andy-wozniewski
Failed to extract player types - No player types found.
Successfully scraped facts for Andy Wozniewsk

In [37]:
# Fetch 1500 to 1550 players' facts
curr_players_metadata = nhl_players_metadata[1500:1550]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Jean-François Jacques at https://www.eliteprospects.com/player/9026/jean-francois-jacques
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Jean-François Jacques
Failed rate: 0.00
Sleep for 1.76 minutes to prevent getting blocked

 [2] Collecting facts for Dan Smith at https://www.eliteprospects.com/player/11762/dan-smith
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Dan Smith
Failed rate: 0.00
Sleep for 0.38 minutes to prevent getting blocked

 [3] Collecting facts for Jason Ryznar at https://www.eliteprospects.com/player/8921/jason-ryznar
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Jason Ryznar
Failed rate: 0.00
Sleep for 1.80 minutes to prevent getting blocked

 [4] Collecting facts for Jean-Philippe Côté at https://www.eliteprospects.com/player/9123/jean-philippe-cote


In [38]:
# Fetch 1550 to 1600 players' facts
curr_players_metadata = nhl_players_metadata[1550:1600]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Jussi Timonen at https://www.eliteprospects.com/player/3193/jussi-timonen
Successfully scraped facts for Jussi Timonen
Failed rate: 0.00
Sleep for 0.63 minutes to prevent getting blocked

 [2] Collecting facts for Marc Methot at https://www.eliteprospects.com/player/10993/marc-methot
Failed to extract player types - No player types found.
Successfully scraped facts for Marc Methot
Failed rate: 0.00
Sleep for 1.06 minutes to prevent getting blocked

 [3] Collecting facts for Chris Conner at https://www.eliteprospects.com/player/10995/chris-conner
Failed to extract player types - No player types found.
Successfully scraped facts for Chris Conner
Failed rate: 0.00
Sleep for 1.09 minutes to prevent getting blocked

 [4] Collecting facts for Janis Sprukts at https://www.eliteprospects.com/player/7420/janis-sprukts
Description not found.
Successfully scraped facts for Janis Sprukts
Failed rate: 0.00
Sleep for 0.45 minutes to prevent getting blocked

 [5] Collecting

In [6]:
# Scrape 1600 to 1650 players' facts
curr_players_metadata = nhl_players_metadata[1600:1650]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Rob Schremp at https://www.eliteprospects.com/player/8876/rob-schremp
Successfully scraped facts for Rob Schremp
Failed rate: 0.00
Sleep for 1.42 minutes to prevent getting blocked

 [2] Collecting facts for Tomi Mäki at https://www.eliteprospects.com/player/3215/tomi-maki
Description not found.
Successfully scraped facts for Tomi Mäki
Failed rate: 0.00
Sleep for 1.55 minutes to prevent getting blocked

 [3] Collecting facts for Dave Bolland at https://www.eliteprospects.com/player/9193/dave-bolland
Description not found.
Successfully scraped facts for Dave Bolland
Failed rate: 0.00
Sleep for 1.45 minutes to prevent getting blocked

 [4] Collecting facts for Ryan Parent at https://www.eliteprospects.com/player/9184/ryan-parent
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Ryan Parent
Failed rate: 0.00
Sleep for 0.55 minutes to prevent getting blocked

 [5] Collecting facts for Aaron Rome at https

In [7]:
# Scrape 1650 to 1700 players' facts
curr_players_metadata = nhl_players_metadata[1650:1700]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for David Perron at https://www.eliteprospects.com/player/11123/david-perron
Successfully scraped facts for David Perron
Failed rate: 0.00
Sleep for 0.25 minutes to prevent getting blocked

 [2] Collecting facts for Sergei Kostitsyn at https://www.eliteprospects.com/player/9653/sergei-kostitsyn
Failed to extract player types - No player types found.
Successfully scraped facts for Sergei Kostitsyn
Failed rate: 0.00
Sleep for 1.82 minutes to prevent getting blocked

 [3] Collecting facts for Milan Lucic at https://www.eliteprospects.com/player/11104/milan-lucic
Successfully scraped facts for Milan Lucic
Failed rate: 0.00
Sleep for 1.24 minutes to prevent getting blocked

 [4] Collecting facts for Daniel Winnik at https://www.eliteprospects.com/player/11480/daniel-winnik
Successfully scraped facts for Daniel Winnik
Failed rate: 0.00
Sleep for 1.73 minutes to prevent getting blocked

 [5] Collecting facts for Matt Niskanen at https://www.eliteprospects.com/player/921

In [8]:
# Scrape 1700 to 1750 players' facts
curr_players_metadata = nhl_players_metadata[1700:1750]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Shawn Matthias at https://www.eliteprospects.com/player/9345/shawn-matthias
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Shawn Matthias
Failed rate: 0.00
Sleep for 0.19 minutes to prevent getting blocked

 [2] Collecting facts for Jordan Smotherman at https://www.eliteprospects.com/player/11568/jordan-smotherman
Failed to extract player types - No player types found.
Successfully scraped facts for Jordan Smotherman
Failed rate: 0.00
Sleep for 0.47 minutes to prevent getting blocked

 [3] Collecting facts for Clay Wilson at https://www.eliteprospects.com/player/11404/clay-wilson
Failed to extract player types - No player types found.
Successfully scraped facts for Clay Wilson
Failed rate: 0.00
Sleep for 1.58 minutes to prevent getting blocked

 [4] Collecting facts for Chris Minard at https://www.eliteprospects.com/player/11349/chris-minard
Failed to extract player types - No player types found.


In [9]:
# Scrape 1750 to 1800 players' facts
curr_players_metadata = nhl_players_metadata[1750:1800]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Claude Giroux at https://www.eliteprospects.com/player/10437/claude-giroux
Failed to extract player types - No player types found.
Successfully scraped facts for Claude Giroux
Failed rate: 0.00
Sleep for 1.18 minutes to prevent getting blocked

 [2] Collecting facts for Adam Berti at https://www.eliteprospects.com/player/9493/adam-berti
Successfully scraped facts for Adam Berti
Failed rate: 0.00
Sleep for 0.76 minutes to prevent getting blocked

 [3] Collecting facts for Justin Abdelkader at https://www.eliteprospects.com/player/10460/justin-abdelkader
Successfully scraped facts for Justin Abdelkader
Failed rate: 0.00
Sleep for 1.57 minutes to prevent getting blocked

 [4] Collecting facts for Marc-André Gragnani at https://www.eliteprospects.com/player/9356/marc-andre-gragnani
Failed to extract player types - No player types found.
Successfully scraped facts for Marc-André Gragnani
Failed rate: 0.00
Sleep for 0.44 minutes to prevent getting blocked

 [5] Col

In [10]:
# Scrape 1800 to 1850 players' facts
curr_players_metadata = nhl_players_metadata[1800:1850]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Luca Sbisa at https://www.eliteprospects.com/player/13946/luca-sbisa
Failed to extract player types - No player types found.
Successfully scraped facts for Luca Sbisa
Failed rate: 0.00
Sleep for 1.54 minutes to prevent getting blocked

 [2] Collecting facts for Jamie McGinn at https://www.eliteprospects.com/player/9336/jamie-mcginn
Failed to extract player types - No player types found.
Successfully scraped facts for Jamie McGinn
Failed rate: 0.00
Sleep for 1.29 minutes to prevent getting blocked

 [3] Collecting facts for Chris Butler at https://www.eliteprospects.com/player/9206/chris-butler
Failed to extract player types - No player types found.
Successfully scraped facts for Chris Butler
Failed rate: 0.00
Sleep for 1.61 minutes to prevent getting blocked

 [4] Collecting facts for Martins Karsums at https://www.eliteprospects.com/player/9238/martins-karsums
Failed to extract player types - No player types found.
Description not found.
Successfully scraped

In [11]:
# Scrape 1850 to 1900 players' facts
curr_players_metadata = nhl_players_metadata[1850:1900]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Warren Peters at https://www.eliteprospects.com/player/11744/warren-peters
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Warren Peters
Failed rate: 0.00
Sleep for 1.60 minutes to prevent getting blocked

 [2] Collecting facts for Matt Halischuk at https://www.eliteprospects.com/player/12560/matt-halischuk
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Matt Halischuk
Failed rate: 0.00
Sleep for 0.74 minutes to prevent getting blocked

 [3] Collecting facts for Brian Salcido at https://www.eliteprospects.com/player/11406/brian-salcido
Failed to extract player types - No player types found.
Successfully scraped facts for Brian Salcido
Failed rate: 0.00
Sleep for 1.88 minutes to prevent getting blocked

 [4] Collecting facts for Jakub Petruzalek at https://www.eliteprospects.com/player/9617/jakub-petruzalek
Successfully scraped facts for 

In [12]:
# Scrape 1900 to 1950 players' facts
curr_players_metadata = nhl_players_metadata[1900:1950]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Andrew MacDonald at https://www.eliteprospects.com/player/12406/andrew-macdonald
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Andrew MacDonald
Failed rate: 0.00
Sleep for 1.18 minutes to prevent getting blocked

 [2] Collecting facts for Jay Beagle at https://www.eliteprospects.com/player/14825/jay-beagle
Description not found.
Successfully scraped facts for Jay Beagle
Failed rate: 0.00
Sleep for 1.89 minutes to prevent getting blocked

 [3] Collecting facts for Per Ledin at https://www.eliteprospects.com/player/186/per-ledin
Successfully scraped facts for Per Ledin
Failed rate: 0.00
Sleep for 0.20 minutes to prevent getting blocked

 [4] Collecting facts for Wes O'Neill at https://www.eliteprospects.com/player/9480/wes-o-neill
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Wes O'Neill
Failed rate: 0.00
Sleep for 1.03 minutes to prev

In [13]:
# Scrape 1950 to 2000 players' facts
curr_players_metadata = nhl_players_metadata[1950:2000]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for John Carlson at https://www.eliteprospects.com/player/18590/john-carlson
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for John Carlson
Failed rate: 0.00
Sleep for 1.32 minutes to prevent getting blocked

 [2] Collecting facts for Frazer McLaren at https://www.eliteprospects.com/player/14501/frazer-mclaren
Description not found.
Successfully scraped facts for Frazer McLaren
Failed rate: 0.00
Sleep for 1.53 minutes to prevent getting blocked

 [3] Collecting facts for Benn Ferriero at https://www.eliteprospects.com/player/9378/benn-ferriero
Failed to extract player types - No player types found.
Successfully scraped facts for Benn Ferriero
Failed rate: 0.00
Sleep for 1.32 minutes to prevent getting blocked

 [4] Collecting facts for Tom Pyatt at https://www.eliteprospects.com/player/9199/tom-pyatt
Failed to extract player types - No player types found.
Successfully scraped facts for Tom Pyatt
Failed r

In [14]:
# Scrape 2000 to 2050 players' facts
curr_players_metadata = nhl_players_metadata[2000:2050]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Eric Tangradi at https://www.eliteprospects.com/player/12514/eric-tangradi
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Eric Tangradi
Failed rate: 0.00
Sleep for 1.83 minutes to prevent getting blocked

 [2] Collecting facts for Maxim Noreau at https://www.eliteprospects.com/player/12153/maxim-noreau
Failed to extract player types - No player types found.
Successfully scraped facts for Maxim Noreau
Failed rate: 0.00
Sleep for 1.76 minutes to prevent getting blocked

 [3] Collecting facts for Andrei Loktionov at https://www.eliteprospects.com/player/11867/andrei-loktionov
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Andrei Loktionov
Failed rate: 0.00
Sleep for 0.82 minutes to prevent getting blocked

 [4] Collecting facts for Marc-André Cliche at https://www.eliteprospects.com/player/10435/marc-andre-cliche
Failed to extract player 

In [15]:
# Scrape 2050 to 2100 players' facts
curr_players_metadata = nhl_players_metadata[2050:2100]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Magnus Pääjärvi at https://www.eliteprospects.com/player/6014/magnus-paajarvi
Successfully scraped facts for Magnus Pääjärvi
Failed rate: 0.00
Sleep for 1.26 minutes to prevent getting blocked

 [2] Collecting facts for Marcus Johansson at https://www.eliteprospects.com/player/6225/marcus-johansson
Failed to extract player types - No player types found.
Successfully scraped facts for Marcus Johansson
Failed rate: 0.00
Sleep for 0.43 minutes to prevent getting blocked

 [3] Collecting facts for Linus Omark at https://www.eliteprospects.com/player/4008/linus-omark
Successfully scraped facts for Linus Omark
Failed rate: 0.00
Sleep for 1.57 minutes to prevent getting blocked

 [4] Collecting facts for Travis Hamonic at https://www.eliteprospects.com/player/17519/travis-hamonic
Successfully scraped facts for Travis Hamonic
Failed rate: 0.00
Sleep for 1.01 minutes to prevent getting blocked

 [5] Collecting facts for Mats Zuccarello at https://www.eliteprospects.co

In [16]:
# Scrape 2100 to 2150 players' facts
curr_players_metadata = nhl_players_metadata[2100:2150]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Joe Vitale at https://www.eliteprospects.com/player/15069/joe-vitale
Successfully scraped facts for Joe Vitale
Failed rate: 0.00
Sleep for 0.76 minutes to prevent getting blocked

 [2] Collecting facts for Aaron Volpatti at https://www.eliteprospects.com/player/39059/aaron-volpatti
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Aaron Volpatti
Failed rate: 0.00
Sleep for 1.63 minutes to prevent getting blocked

 [3] Collecting facts for Patrice Cormier at https://www.eliteprospects.com/player/19042/patrice-cormier
Failed to extract player types - No player types found.
Successfully scraped facts for Patrice Cormier
Failed rate: 0.00
Sleep for 1.50 minutes to prevent getting blocked

 [4] Collecting facts for Tomas Vincour at https://www.eliteprospects.com/player/12565/tomas-vincour
Failed to extract player types - No player types found.
Successfully scraped facts for Tomas Vincour
Failed rate: 0.00

In [17]:
# Scrape 2150 to 2200 players' facts
curr_players_metadata = nhl_players_metadata[2150:2200]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Hugh Jessiman at https://www.eliteprospects.com/player/11748/hugh-jessiman
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Hugh Jessiman
Failed rate: 0.00
Sleep for 1.22 minutes to prevent getting blocked

 [2] Collecting facts for Travis Morin at https://www.eliteprospects.com/player/11446/travis-morin
Failed to extract player types - No player types found.
Successfully scraped facts for Travis Morin
Failed rate: 0.00
Sleep for 1.19 minutes to prevent getting blocked

 [3] Collecting facts for Nick Drazenovic at https://www.eliteprospects.com/player/11637/nick-drazenovic
Failed to extract player types - No player types found.
Successfully scraped facts for Nick Drazenovic
Failed rate: 0.00
Sleep for 1.66 minutes to prevent getting blocked

 [4] Collecting facts for Andreas Engqvist at https://www.eliteprospects.com/player/6626/andreas-engqvist
Failed to extract player types - No player types found

In [18]:
# Scrape 2200 to 2250 players' facts
curr_players_metadata = nhl_players_metadata[2200:2250]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Louis Leblanc at https://www.eliteprospects.com/player/31615/louis-leblanc
Failed to extract player types - No player types found.
Successfully scraped facts for Louis Leblanc
Failed rate: 0.00
Sleep for 0.20 minutes to prevent getting blocked

 [2] Collecting facts for Zack Kassian at https://www.eliteprospects.com/player/15390/zack-kassian
Description not found.
Successfully scraped facts for Zack Kassian
Failed rate: 0.00
Sleep for 0.73 minutes to prevent getting blocked

 [3] Collecting facts for David Savard at https://www.eliteprospects.com/player/32878/david-savard
Successfully scraped facts for David Savard
Failed rate: 0.00
Sleep for 1.05 minutes to prevent getting blocked

 [4] Collecting facts for Jimmy Hayes at https://www.eliteprospects.com/player/11463/jimmy-hayes
Failed to extract player types - No player types found.
Successfully scraped facts for Jimmy Hayes
Failed rate: 0.00
Sleep for 1.89 minutes to prevent getting blocked

 [5] Collecting 

In [19]:
# Scrape 2250 to 2300 players' facts
curr_players_metadata = nhl_players_metadata[2250:2300]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Cade Fairchild at https://www.eliteprospects.com/player/9309/cade-fairchild
[ERROR] Failed to get facts for Cade Fairchild: HTTPConnectionPool(host='localhost', port=63072): Read timed out. (read timeout=120)
Successfully scraped facts for Cade Fairchild
Failed rate: 0.00
Sleep for 1.74 minutes to prevent getting blocked

 [2] Collecting facts for Riley Nash at https://www.eliteprospects.com/player/12507/riley-nash
Failed to extract player types - No player types found.
Successfully scraped facts for Riley Nash
Failed rate: 0.00
Sleep for 0.96 minutes to prevent getting blocked

 [3] Collecting facts for Mika Zibanejad at https://www.eliteprospects.com/player/23856/mika-zibanejad
Successfully scraped facts for Mika Zibanejad
Failed rate: 0.00
Sleep for 0.19 minutes to prevent getting blocked

 [4] Collecting facts for Colten Teubert at https://www.eliteprospects.com/player/13625/colten-teubert
Successfully scraped facts for Colten Teubert
Failed rate: 0.00
Sl

In [20]:
# Scrape 2300 to 2350 players' facts
curr_players_metadata = nhl_players_metadata[2300:2350]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Kevin Marshall at https://www.eliteprospects.com/player/12508/kevin-marshall
Failed to extract player types - No player types found.
Successfully scraped facts for Kevin Marshall
Failed rate: 0.00
Sleep for 1.39 minutes to prevent getting blocked

 [2] Collecting facts for Carter Ashton at https://www.eliteprospects.com/player/21033/carter-ashton
Successfully scraped facts for Carter Ashton
Failed rate: 0.00
Sleep for 0.21 minutes to prevent getting blocked

 [3] Collecting facts for Brandon Bollig at https://www.eliteprospects.com/player/39199/brandon-bollig
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Brandon Bollig
Failed rate: 0.00
Sleep for 1.84 minutes to prevent getting blocked

 [4] Collecting facts for Nail Yakupov at https://www.eliteprospects.com/player/21228/nail-yakupov
Failed to extract player types - No player types found.
Successfully scraped facts for Nail Yakupov
Failed rate: 0

In [21]:
# Scrape 2350 to 2400 players' facts
curr_players_metadata = nhl_players_metadata[2350:2400]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Dave Dziurzynski at https://www.eliteprospects.com/player/45871/dave-dziurzynski
Failed to extract player types - No player types found.
Successfully scraped facts for Dave Dziurzynski
Failed rate: 0.00
Sleep for 0.58 minutes to prevent getting blocked

 [2] Collecting facts for Anders Lee at https://www.eliteprospects.com/player/19068/anders-lee
Description not found.
Successfully scraped facts for Anders Lee
Failed rate: 0.00
Sleep for 1.78 minutes to prevent getting blocked

 [3] Collecting facts for Matt Tennyson at https://www.eliteprospects.com/player/37796/matt-tennyson
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Matt Tennyson
Failed rate: 0.00
Sleep for 1.87 minutes to prevent getting blocked

 [4] Collecting facts for Anthony Peluso at https://www.eliteprospects.com/player/15038/anthony-peluso
Failed to extract player types - No player types found.
Description not found.
Successfully s

In [22]:
# Scrape 2400 to 2450 players' facts
curr_players_metadata = nhl_players_metadata[2400:2450]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Jon Rheault at https://www.eliteprospects.com/player/14861/jon-rheault
Failed to extract player types - No player types found.
Successfully scraped facts for Jon Rheault
Failed rate: 0.00
Sleep for 0.44 minutes to prevent getting blocked

 [2] Collecting facts for Derek Grant at https://www.eliteprospects.com/player/18242/derek-grant
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Derek Grant
Failed rate: 0.00
Sleep for 0.27 minutes to prevent getting blocked

 [3] Collecting facts for Scott Laughton at https://www.eliteprospects.com/player/76690/scott-laughton
Failed to extract player types - No player types found.
Successfully scraped facts for Scott Laughton
Failed rate: 0.00
Sleep for 1.77 minutes to prevent getting blocked

 [4] Collecting facts for Chris Brown at https://www.eliteprospects.com/player/31273/chris-brown
Failed to extract player types - No player types found.
Description not fou

In [23]:
# Scrape 2450 to 2500 players' facts
curr_players_metadata = nhl_players_metadata[2450:2500]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Linden Vey at https://www.eliteprospects.com/player/28422/linden-vey
Failed to extract player types - No player types found.
Successfully scraped facts for Linden Vey
Failed rate: 0.00
Sleep for 1.95 minutes to prevent getting blocked

 [2] Collecting facts for Rasmus Ristolainen at https://www.eliteprospects.com/player/43497/rasmus-ristolainen
Description not found.
Successfully scraped facts for Rasmus Ristolainen
Failed rate: 0.00
Sleep for 1.14 minutes to prevent getting blocked

 [3] Collecting facts for Luke Gazdic at https://www.eliteprospects.com/player/15037/luke-gazdic
Failed to extract player types - No player types found.
Successfully scraped facts for Luke Gazdic
Failed rate: 0.00
Sleep for 0.56 minutes to prevent getting blocked

 [4] Collecting facts for Colton Sissons at https://www.eliteprospects.com/player/48393/colton-sissons
[ERROR] Failed to get facts for Colton Sissons: HTTPConnectionPool(host='localhost', port=53063): Read timed out. (r

In [24]:
# Scrape 2500 to 2600 players' facts
curr_players_metadata = nhl_players_metadata[2500:2600]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Petter Granberg at https://www.eliteprospects.com/player/13901/petter-granberg
Successfully scraped facts for Petter Granberg
Failed rate: 0.00
Sleep for 0.92 minutes to prevent getting blocked

 [2] Collecting facts for Julian Melchiori at https://www.eliteprospects.com/player/73778/julian-melchiori
Failed to extract player types - No player types found.
Successfully scraped facts for Julian Melchiori
Failed rate: 0.00
Sleep for 1.74 minutes to prevent getting blocked

 [3] Collecting facts for Ryan Sproul at https://www.eliteprospects.com/player/62407/ryan-sproul
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Ryan Sproul
Failed rate: 0.00
Sleep for 0.46 minutes to prevent getting blocked

 [4] Collecting facts for Cody Kunyk at https://www.eliteprospects.com/player/44471/cody-kunyk
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Cody 

In [25]:
# Scrape 2600 to 2700 players' facts
curr_players_metadata = nhl_players_metadata[2600:2700]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Tim Schaller at https://www.eliteprospects.com/player/38059/tim-schaller
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Tim Schaller
Failed rate: 0.00
Sleep for 0.82 minutes to prevent getting blocked

 [2] Collecting facts for Petr Straka at https://www.eliteprospects.com/player/34805/petr-straka
Failed to extract player types - No player types found.
Successfully scraped facts for Petr Straka
Failed rate: 0.00
Sleep for 0.65 minutes to prevent getting blocked

 [3] Collecting facts for David Musil at https://www.eliteprospects.com/player/23298/david-musil
Failed to extract player types - No player types found.
Successfully scraped facts for David Musil
Failed rate: 0.00
Sleep for 1.26 minutes to prevent getting blocked

 [4] Collecting facts for Danny Biega at https://www.eliteprospects.com/player/40194/danny-biega
Failed to extract player types - No player types found.
Successfully scraped fact

In [26]:
# Scrape 2700 to 2800 players' facts
curr_players_metadata = nhl_players_metadata[2700:2800]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Andreas Martinsen at https://www.eliteprospects.com/player/11958/andreas-martinsen
Failed to extract player types - No player types found.
Successfully scraped facts for Andreas Martinsen
Failed rate: 0.00
Sleep for 1.86 minutes to prevent getting blocked

 [2] Collecting facts for Conor Sheary at https://www.eliteprospects.com/player/94484/conor-sheary
Failed to extract player types - No player types found.
Successfully scraped facts for Conor Sheary
Failed rate: 0.00
Sleep for 1.57 minutes to prevent getting blocked

 [3] Collecting facts for Daniel Carr at https://www.eliteprospects.com/player/39435/daniel-carr
Failed to extract player types - No player types found.
Successfully scraped facts for Daniel Carr
Failed rate: 0.00
Sleep for 0.97 minutes to prevent getting blocked

 [4] Collecting facts for Dennis Rasmussen at https://www.eliteprospects.com/player/9985/dennis-rasmussen
Failed to extract player types - No player types found.
Successfully scraped 

In [27]:
# Scrape 2800 to 2900 players' facts
curr_players_metadata = nhl_players_metadata[2800:2900]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Mike Matheson at https://www.eliteprospects.com/player/76333/mike-matheson
Failed to extract player types - No player types found.
Successfully scraped facts for Mike Matheson
Failed rate: 0.00
Sleep for 1.48 minutes to prevent getting blocked

 [2] Collecting facts for Kurtis Gabriel at https://www.eliteprospects.com/player/92683/kurtis-gabriel
Description not found.
Successfully scraped facts for Kurtis Gabriel
Failed rate: 0.00
Sleep for 1.15 minutes to prevent getting blocked

 [3] Collecting facts for Markus Hännikäinen at https://www.eliteprospects.com/player/45426/markus-hannikainen
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Markus Hännikäinen
Failed rate: 0.00
Sleep for 1.21 minutes to prevent getting blocked

 [4] Collecting facts for Esa Lindell at https://www.eliteprospects.com/player/50019/esa-lindell
Description not found.
Successfully scraped facts for Esa Lindell
Failed rate: 0.

In [28]:
# Scrape 2900 to 3000 players' facts
curr_players_metadata = nhl_players_metadata[2900:3000]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Wade Megan at https://www.eliteprospects.com/player/38054/wade-megan
Failed to extract player types - No player types found.
Successfully scraped facts for Wade Megan
Failed rate: 0.00
Sleep for 1.28 minutes to prevent getting blocked

 [2] Collecting facts for Danny O'Regan at https://www.eliteprospects.com/player/121936/danny-o-regan
Failed to extract player types - No player types found.
Successfully scraped facts for Danny O'Regan
Failed rate: 0.00
Sleep for 1.44 minutes to prevent getting blocked

 [3] Collecting facts for Tyson Jost at https://www.eliteprospects.com/player/148111/tyson-jost
Successfully scraped facts for Tyson Jost
Failed rate: 0.00
Sleep for 1.70 minutes to prevent getting blocked

 [4] Collecting facts for Matt Lorito at https://www.eliteprospects.com/player/62740/matt-lorito
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Matt Lorito
Failed rate: 0.00
Sleep for 1.82 minute

In [29]:
# Scrape 3000 to 3100 players' facts
curr_players_metadata = nhl_players_metadata[3000:3100]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Tanner Fritz at https://www.eliteprospects.com/player/23940/tanner-fritz
Failed to extract player types - No player types found.
Successfully scraped facts for Tanner Fritz
Failed rate: 0.00
Sleep for 0.46 minutes to prevent getting blocked

 [2] Collecting facts for Victor Mete at https://www.eliteprospects.com/player/232703/victor-mete
Failed to extract player types - No player types found.
Successfully scraped facts for Victor Mete
Failed rate: 0.00
Sleep for 1.68 minutes to prevent getting blocked

 [3] Collecting facts for Zach Aston-Reese at https://www.eliteprospects.com/player/98068/zach-aston-reese
Failed to extract player types - No player types found.
Description not found.
Successfully scraped facts for Zach Aston-Reese
Failed rate: 0.00
Sleep for 1.53 minutes to prevent getting blocked

 [4] Collecting facts for Oskar Lindblom at https://www.eliteprospects.com/player/86181/oskar-lindblom
Successfully scraped facts for Oskar Lindblom
Failed rate: 

In [ ]:
# Scrape 3100 to 3200 players' facts
curr_players_metadata = nhl_players_metadata[3100:3200]
get_players_stats_by_batch(curr_players_metadata)

## Check Missing Players

In [10]:
# Get unique players from nhl_players_facts.csv
if os.path.exists(output_path):
    # Read in the CSV file and set low_memory = False
    nhl_players_facts = pd.read_csv(output_path, low_memory=False)
    unique_players = set(nhl_players_facts['player_name'].unique())
else:
    unique_players = set()

In [11]:
# Find players in nhl_players_metadata 0-1500 not in unique_players
subset = nhl_players_metadata[0:1600]
players_to_scrape = subset[~subset['player_name'].isin(unique_players)].reset_index(drop=True)
players_to_scrape

,player_name,fw_def,link


In [ ]:
# Fetch Missing Players' Facts
get_players_stats_by_batch(players_to_scrape)